In [2]:
import cv2
import dlib
import time
import os
import numpy as np
import pygame  
from imutils import face_utils
from scipy.spatial import distance
from pushbullet import Pushbullet  
import geocoder  
from datetime import datetime
import pandas as pd


pygame.mixer.init()
pygame.mixer.music.load('audio/alert.wav')


EYE_ASPECT_RATIO_THRESHOLD = 0.3


EYE_ASPECT_RATIO_CONSEC_FRAMES = 20


COUNTER = 0


initial_image_captured = False
initial_image_name = None


face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

API_KEY = 'o.RnIDtF8yXUoAzRaNXxtvaDfeDrfwyRON'  
pb = Pushbullet(API_KEY)
notification_sent = False


video_capture = cv2.VideoCapture(0)


time.sleep(2)


alert_images_directory = "alert_images"


if not os.path.exists(alert_images_directory):
    os.makedirs(alert_images_directory)


initial_images_directory = "initial_images"


if not os.path.exists(initial_images_directory):
    os.makedirs(initial_images_directory)

 
excel_file = "drowsiness_data.xlsx"

 
if not os.path.exists(excel_file):
    df = pd.DataFrame(columns=["Timestamp", "Initial Image", "Alert Image", "Location"])
    df.to_excel(excel_file, index=False)

while True:
     
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

   
    faces = detector(gray, 0)

    
    face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

 
    if len(face_rectangle) > 0 and not initial_image_captured:
        initial_image_name = os.path.join(initial_images_directory, f"initial_image_{datetime.now().strftime('%Y%m%d%H%M%S')}.png")
        cv2.imwrite(initial_image_name, frame)
        initial_image_captured = True

    
    for (x, y, w, h) in face_rectangle:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    
    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

 
        leftEyeAspectRatio = eye_aspect_ratio(leftEye)
        rightEyeAspectRatio = eye_aspect_ratio(rightEye)

        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

         
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

     
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        
        if eyeAspectRatio < EYE_ASPECT_RATIO_THRESHOLD:
            COUNTER += 1
            
            if COUNTER >= EYE_ASPECT_RATIO_CONSEC_FRAMES:
                pygame.mixer.music.play(-1)
                cv2.putText(frame, "Alert", (150, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
                if not notification_sent:
                     
                    g = geocoder.ip('me')
                    location = g.latlng
                    if location:
                        location_str = f"Location: {location[0]}, {location[1]}"
                    else:
                        location_str = "Location: Unable to determine"

                  
                    pb.push_note("Drowsiness Alert", f"The driver appears to be drowsy.\n{location_str}")
                    notification_sent = True
                    
                  
                    alert_image_name = f"{alert_images_directory}/alert_{datetime.now().strftime('%Y%m%d%H%M%S')}.png"
                    cv2.imwrite(alert_image_name, frame)

                 
                    try:
                        df = pd.read_excel(excel_file)
                        new_data = pd.DataFrame([{
                            "Timestamp": timestamp,
                            "Initial Image": initial_image_name,
                            "Alert Image": alert_image_name,
                            "Location": location_str
                        }])
                        df = pd.concat([df, new_data], ignore_index=True)
                        df.to_excel(excel_file, index=False)
                    except PermissionError as e:
                        print(f"Permission error: {e}")
                        pygame.mixer.music.stop()
                        COUNTER = 0
                        notification_sent = False

        else:
            pygame.mixer.music.stop()
            COUNTER = 0
            notification_sent = False


    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video_capture.release()
cv2.destroyAllWindows()
